In [1]:
%run ml_header.py

In [2]:
%run get_data.py
benchmarks = ('adpcm_encoder', 'aes', 'ann', 'average', 'decimation', 'fft_fixed', 'fir',
              'idct', 'interpolation', 'kasumi', 'qsort', 'snow3g', 'sobel')  # 13
gd = GetData('data/ASIC-2-FPGA', benchmarks)
gd.main()

In [3]:
%run methods.py

In [4]:
%run machine_learning.py

In [5]:
import itertools

In [6]:
list(itertools.combinations(benchmarks, 12))

[('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'idct',
  'interpolation',
  'kasumi',
  'qsort',
  'snow3g'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'idct',
  'interpolation',
  'kasumi',
  'qsort',
  'sobel'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'idct',
  'interpolation',
  'kasumi',
  'snow3g',
  'sobel'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'idct',
  'interpolation',
  'qsort',
  'snow3g',
  'sobel'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'idct',
  'kasumi',
  'qsort',
  'snow3g',
  'sobel'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed',
  'fir',
  'interpolation',
  'kasumi',
  'qsort',
  'snow3g',
  'sobel'),
 ('adpcm_encoder',
  'aes',
  'ann',
  'average',
  'decimation',
  'fft_fixed'

In [7]:
features = ['AREA', 'state', 'FU', 'REG', 'MUX', 'DEC', 'pin_pair',
            'net', 'max', 'min', 'ave', 'MISC', 'MEM', 'sim', 'Pmax',
            'Pmin', 'Pave', 'Latency', 'BlockMemoryBit', 'DSP', 'Slices']
invalid_features = ['Slices', 'Latency']
valid_features = [i for i in features if i not in invalid_features]
label = 'Slices'

In [8]:
def execute(df, estimator, kind):
    # remove attribute columns
    data = df[features].copy()
    # fix missing data
    data = ML.fix_missing_data(data)
    # X, y
    X, y = ML.separate_feature_label(data, valid_features=valid_features, label=label)
    # feature scaling
    X = ML.feature_scaling(X)
    if kind == 'training':
        estimator.fit(X, y)
        return estimator
    elif kind == 'testing':
        y_pred = estimator.predict(X)
        data['AREA'] = y_pred
        return DirectMapping.main(data)

In [9]:
estimator = execute(gd.data_v4['adpcm_encoder'], estimators[0], kind='training')
scores = execute(gd.data_v4['average'], estimator, kind='testing')
scores

{'adrs_ave': 0.0,
 'adrs_ave_rms': 0.0,
 'adrs_max': 0.0,
 'adrs_max_rms': 0.0,
 'cardinality': 3,
 'dominance': 1.0,
 'hypervolume': 0.43148731064332085}

In [10]:
estimator.coef_

array([  123.3893,   -19.6479,   120.7541,    18.8534,    -5.8628,
          -0.    ,  1292.9906, -1136.0835,   -19.6479,   -19.6479,
         -19.6479,    83.4866,     0.    ,     0.    ,     0.    ,
           0.    ,     0.    ,     0.    ,     0.    ])

In [16]:
%%time
scores_1 = dict()
for benchmarks_train in itertools.combinations(benchmarks, 1):
    benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
    data_train = pd.concat([gd.data_v4[i] for i in benchmarks_train], axis=0, ignore_index=True)
    estimator = execute(data_train, estimators[0], kind='training')
    key = ';'.join(list(benchmarks_train))
    scores_1[key] = dict()
    for benchmark_test in benchmarks_test:
        scores_1[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')

Wall time: 20.6 s


In [18]:
%%time
scores_2 = dict()
for benchmarks_train in itertools.combinations(benchmarks, 2):
    benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
    data_train = pd.concat([gd.data_v4[i] for i in benchmarks_train], axis=0, ignore_index=True)
    estimator = execute(data_train, estimators[0], kind='training')
    key = ';'.join(list(benchmarks_train))
    scores_2[key] = dict()
    for benchmark_test in benchmarks_test:
        scores_2[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')

Wall time: 1min 54s


In [21]:
%%time
scores_v4 = dict()
for benchmarks_cnt in range(1, len(benchmarks)):
    scores = dict()
    for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
        benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
        data_train = pd.concat([gd.data_v4[i] for i in benchmarks_train], axis=0, ignore_index=True)
        estimator = execute(data_train, estimators[0], kind='training')
        key = ';'.join(list(benchmarks_train))
        scores[key] = dict()
        for benchmark_test in benchmarks_test:
            scores[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')
    scores_v4[str(benchmarks_cnt)] = scores

Wall time: 2h 3min 44s


In [56]:
%%time
scores_v5 = dict()
for benchmarks_cnt in range(1, len(benchmarks)):
    scores = dict()
    for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
        benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
        data_train = pd.concat([gd.data_v5[i] for i in benchmarks_train], axis=0, ignore_index=True)
        estimator = execute(data_train, estimators[0], kind='training')
        key = ';'.join(list(benchmarks_train))
        scores[key] = dict()
        for benchmark_test in benchmarks_test:
            scores[key][benchmark_test] = execute(gd.data_v5[benchmark_test], estimator, kind='testing')
    scores_v5[str(benchmarks_cnt)] = scores

Wall time: 2h 6min 39s


In [13]:
with open('results_v4.csv', 'w') as f:
    f.write('train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality\n')
    for bench_cnt in scores_v4_rf.keys():
        for training, testing in scores_v4_rf[bench_cnt].items():
            for k, v in testing.items():
                f.write(training + ',' + k + ',' + ','.join([f'{i:.4f}' for i in v.values()]) + '\n')

In [69]:
with open('results_v5.csv', 'w') as f:
    f.write('train,test,adrs_ave,adrs_max,adrs_ave_rms,adrs_max_rms,hypervolume,dominance,cardinality\n')
    for bench_cnt in scores_v5_ada.keys():
        for training, testing in scores_v5[bench_cnt].items():
            for k, v in testing.items():
                f.write(training + ',' + k + ',' + ','.join([f'{i:.4f}' for i in v.values()]) + '\n')

In [65]:
estimators[-2]

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=None)

In [66]:
%%time
scores_v4_ada = dict()
for benchmarks_cnt in range(1, len(benchmarks)):
    scores = dict()
    for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
        benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
        data_train = pd.concat([gd.data_v4[i] for i in benchmarks_train], axis=0, ignore_index=True)
        estimator = execute(data_train, estimators[-2], kind='training')
        key = ';'.join(list(benchmarks_train))
        scores[key] = dict()
        for benchmark_test in benchmarks_test:
            scores[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')
    scores_v4_ada[str(benchmarks_cnt)] = scores

Wall time: 17h 9min 50s


In [67]:
%%time
scores_v5_ada = dict()
for benchmarks_cnt in range(1, len(benchmarks)):
    scores = dict()
    for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
        benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
        data_train = pd.concat([gd.data_v5[i] for i in benchmarks_train], axis=0, ignore_index=True)
        estimator = execute(data_train, estimators[-2], kind='training')
        key = ';'.join(list(benchmarks_train))
        scores[key] = dict()
        for benchmark_test in benchmarks_test:
            scores[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')
    scores_v5_ada[str(benchmarks_cnt)] = scores

Wall time: 17h 7min 52s


In [11]:
estimator = estimators[7]

In [12]:
%%time
scores_v4_rf = dict()
for benchmarks_cnt in range(1, len(benchmarks)):
    scores = dict()
    for benchmarks_train in itertools.combinations(benchmarks, benchmarks_cnt):
        benchmarks_test = (i for i in benchmarks if i not in benchmarks_train)
        data_train = pd.concat([gd.data_v4[i] for i in benchmarks_train], axis=0, ignore_index=True)
        estimator = execute(data_train, estimators[-2], kind='training')
        key = ';'.join(list(benchmarks_train))
        scores[key] = dict()
        for benchmark_test in benchmarks_test:
            scores[key][benchmark_test] = execute(gd.data_v4[benchmark_test], estimator, kind='testing')
    scores_v4_rf[str(benchmarks_cnt)] = scores

Wall time: 16h 55min 30s
